# Base de données répertoriant les textes législatifs mentionnant les types d'infractions nous intéressants entre 1996 et 2022 en France
---

### Table des matières

* [Récupération des données de Légifrance via une API](#section1)
    * [Installation et importation des modules](#section11)
    * [Requêtes sur l'API](#section12)
    * [Travail sur les fichiers extraits](#section13)
* [Nettoyage des données de Légifrance](#section2)
* [Sauvegarde des tableaux de données finalisées](#section3)

Ce notebook contient les différents codes qui nous ont permis d'accéder aux données disponibles grâce à l'API Piste de Légifrance.

Sur l'API de Légifrance les requêtes doivent être faites sur un fonds, soit un filtre correspondant à une catégorie spécifique de la base de données de Légifrance. Les différents fonds incluent notamment LODA , qui regroupe les lois, les ordonnances, les décrets et les arrêtés, mais il y a aussi CODE pour les documents relatifs aux différents codes et ALL qui permet de faire une requête sur tous les fonds. 

Au début de notre récupération des données nos requêtes ont été effectuées sur le fond LODA, ensuite CODE et enfin ALL, avant de revenir sur LODA. Cet ordre n'est dû qu'à l'absence de documentation claire, au faible nombre de projets présents sur internet utilisant l'API de Légifrance et d'erreurs que l'on n'arrive pas à expliquer et qui nous empêchent d'avoir accès à l'intégralité des fonds. 

En fait, lorsqu'on effectue une requête, l'API ne nous renvoie qu'un nombre limité de résultats (100 maximum), donc il faut faire une boucle pour récupérer tous les documents. Les requêtes supérieures au 10001e éléments renvoient une erreur 503, soit une erreur du serveur. Malgré des recherches et des mails envoyés au support et à des personnes travaillant à l'AIFE (Agence pour l'Informatique Financière de l'État) sur l'API de Légifrance nous n'avons pas trouvé de solution à cela. 

Ainsi, pour l'avancement du projet, nous avons pris la décision de ne récupérer que les données sur le fond LODA, car ce fond est d'après nous le plus pertinent - les modifications des codes étant plus difficiles à étudier sous l'angle quantitatif par rapport aux lois, car il s'agit en partie de modification de vocabulaire employé.

### Récupération des données avec le fond LODA <a class="anchor" id="section1"></a>

##### Installation et importation des modules <a class="anchor" id="section11"></a>

In [ ]:
!pip install -r requirements.txt

In [2]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import requests
import pandas as pd
import math
import json
from datetime import datetime, timedelta
from threading import Thread
import zipfile
import numpy as np 
import re
import matplotlib.pyplot as plt
import s3fs
import shutil

##### Requêtes sur l'API <a class="anchor" id="section12"></a>

Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation, ces derniers seront écrits dans un fichier .env à chaque fois qu'on lance le code pour récupérer un client. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

Le code pour récupérer le token est grandement inspiré de celui proposé par le Gitlab de Piste présent à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads)

In [3]:
def get_client():
    """
    Récupère un client OAuth2Session configuré avec un token d'accès depuis le serveur OAuth. 

    :return: Un objet OAuth2Session prêt à être utilisé pour des requêtes API.
    """
    TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

    # Charger les identifiants client depuis le fichier .env
    load_dotenv()
    client_id = os.getenv("CLIENT_ID")
    client_secret = os.getenv("CLIENT_SECRET")

    # Requête pour obtenir le token
    res = requests.post(
        TOKEN_URL,
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": "openid"
        }
    )

    res.raise_for_status()  # Lever une erreur si la requête échoue

    token = res.json()

    # Retourner un client OAuth2Session configuré
    return OAuth2Session(client_id, token=token)

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données. La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

In [4]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

Les codes ci-dessous sont les requêtes que l'on va envoyer à l'API, ces dernières sont en json et pour en envoyer une il faut taper client.post(api_url, json=code).json(), selon l'endpoint il faudra mettre get à la place de post. 

Les codes ci-dessous permettent de récupérer un ensemble de documents appartenant à l'ensemble des lois, ordonnance, décrets et arrêtés de Légifrance entre le premier janvier 1996 et le 31 août 2022 qui contiennent au moins un des mots d'une liste définie. Cette liste comprend les termes des taux de délinquances détaillés [ici](database_délinquance.ipynb#Calcul-des-taux-de-délinquance). Ce code est un json et sera envoyé par une méthode post à l'API, elle nous renverra un nombre de résultats limités (100 ici). Ce code est très inspiré de celui disponible sur le [site de l'API](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr) à l'endpoint /search.

La seule différence entre les deux codes résident dans le filtre temporel, car comme l'API ne permet pas de renvoyer plus de 10001 résultats par requête alors nous sommes obligés d'en faire deux distinctes et de regrouper les données a posteriori. Cela aurait pu fonctionner sur le fond All en fractionnant encore plus, or nous n'avons pas réussi à faire fonctionner le filtre temporel dans une requête sur All. 

In [5]:
code_api_LODA_1 = {
    "recherche": {
        "filtres": [{"dates": {"start": "2008-01-01", "end": "2022-08-31"}, "facette": "DATE_SIGNATURE"}],
        "sort": "SIGNATURE_DATE_DESC",
        "fromAdvancedRecherche": True,
        "champs": [
            {
                "typeChamp": "ALL",
                "criteres": [
                    {"typeRecherche": "UN_DES_MOTS", "valeur": mot, "operateur": "OU"} for mot in [
                        "délinquance", "crime", "délit", "Homicides", "Vols", "Stupéfiants", "Escroquerie",
                        "Contrefaçon", "Sequestrations", "Recels", "Proxénétisme", "Menaces", "Cambriolages",
                        "infraction", "Attentats", "dégradations", "Outrages"
                    ]
                ],
                "operateur": "OU"
            }
        ],
        "pageSize": 100,
        "pageNumber": 1,
        "operateur": "ET",
        "typePagination": "DEFAUT"
    },
    "fond": "LODA_DATE"
}


In [6]:
code_api_LODA_2 = {
    "recherche": {
        "filtres": [{"dates": {"start": "1996-01-01", "end": "2008-12-31"}, "facette": "DATE_SIGNATURE"}],
        "sort": "SIGNATURE_DATE_DESC",
        "fromAdvancedRecherche": True,
        "champs": [
            {
                "typeChamp": "ALL",
                "criteres": [
                    {"typeRecherche": "UN_DES_MOTS", "valeur": mot, "operateur": "OU"} for mot in [
                        "délinquance", "crime", "délit", "Homicides", "Vols", "Stupéfiants", "Escroquerie",
                        "Contrefaçon", "Sequestrations", "Recels", "Proxénétisme", "Menaces", "Cambriolages",
                        "infraction", "Attentats", "dégradations", "Outrages"
                    ]
                ],
                "operateur": "OU"
            }
        ],
        "pageSize": 100,
        "pageNumber": 1,
        "operateur": "ET",
        "typePagination": "DEFAUT"
    },
    "fond": "LODA_DATE"
}

Comme la limite de résultats est de 100 pour une requête nous avons créer deux fonctions complémentaire, l'une parcourant toutes les pages de notre requêtes afin de récupérer tous les résultats tout en s'appuyant sur la seconde qui permet de les sauvegarder dans un fichier json. 

In [7]:
def save_results_to_file(results, file_name, current_page):
    """
    Sauvegarde les résultats dans un fichier JSON. Si le fichier existe, ajoute les nouvelles données.

    :param results: Liste des résultats à sauvegarder.
    :param file_name: Nom du fichier JSON.
    :param current_page: La page actuelle traitée.
    """
    try:
        # Charger les données existantes si le fichier existe
        with open(file_name, "r", encoding="utf-8") as file:
            existing_data = json.load(file)
            if not isinstance(existing_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            existing_results = existing_data.get("results", [])
            start_page = existing_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        existing_results = []
        start_page = 1

    # Ajouter les nouveaux résultats
    existing_results.extend(results)

    # Sauvegarder les résultats mis à jour avec la page actuelle
    with open(file_name, "w", encoding="utf-8") as file:
        json.dump({"results": existing_results, "current_page": current_page}, file, ensure_ascii=False, indent=4)

In [8]:
def collect_all_results(api_host, code, file_name):
    ''' 
    Récupère tous les résultats relatifs à une requête API. 

    :param api_host: addresse à laquelle envoyer la requête avec le endpoint correspondant 
    :param code: code permettant de faire la requête correspondante à nos recherches, 
    pour l'utilisation de la fonction on mettra de fait 
    collect_all_results(api_host, json = code)
    
    :return: le nombre total de page 
    '''

    # Charger la page de démarrage depuis le fichier de sauvegarde s'il existe comme il faut 
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            saved_data = json.load(file)
            if not isinstance(saved_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            start_page = saved_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        print(f"Fichier {file_name} introuvable ou mal structuré. Démarrage depuis la première page.")
        start_page = 1

    # Initialiser le client OAuth2Session 
    client = get_client()

    # On fixe un temps d'expiration à 55 min car le token dure 60 min
    expires_in = 55*60
    token_expiry = datetime.now() + timedelta(seconds=expires_in) 

    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")
    print(f"Reprise à partir de la page {start_page}")

    # Liste pour stocker les résultats courants
    all_results = []

    # On boucle sur le nombre de pages
    for page_number in range(start_page, total_pages + 1):

        # Vérifier si le token doit être renouvelé
        if datetime.now() >= token_expiry:
            print("Renouvellement du client OAuth...")
            client = get_client()
            token_expiry = datetime.now() + timedelta(seconds=expires_in)
        
        # Récupère le bon numéro de page et lance la requête
        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        page_results = response.get("results", [])

        # Teste si une erreur 503 arrive et arrête la boucle si tel est le cas
        if response.get("error") == 503:
            print(response)
            break

        # Affiche une requête toutes les 10 pour du contrôle 
        if page_number % 10 == 0: 
            print(response)

        # Ajouter les résultats de la page courante
        all_results.extend(page_results)

        # Sauvegarder les résultats toutes les 20 pages ou à la dernière page
        if page_number % 20 == 0 or page_number == total_pages:
            print(f"Ajout des pages jusqu'à la page {page_number} dans {file_name}...")
            save_results_to_file(all_results, file_name, page_number)

            # Réinitialiser la liste des résultats sauvegardés
            all_results = []

    print(f"Récupération terminée. Dernière page sauvegardée : {total_pages}")
    return total_pages


In [ ]:
results_1996_to_2008 = collect_all_results(API_HOST+"/search", code_api_LODA_2, "results_1996_to_2008.json")
results_2008_to_2022 = collect_all_results(API_HOST+"/search", code_api_LODA_1, "results_2008_to_2022.json")

##### Travail sur les fichiers extraits <a class="anchor" id="section13"></a>

In [10]:
def move_files(source_folder, destination_folder, files_to_move):
    """
    Déplace les fichiers spécifiés d'un dossier source à un dossier destination.

    :param source_folder: Chemin du dossier source (str)
    :param destination_folder: Chemin du dossier destination (str)
    :param files_to_move: Liste des noms de fichiers à déplacer (list)
    """
    # Vérifier si le dossier de destination existe, sinon le créer
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        print(f"Dossier de destination créé : {destination_folder}")
    
    # Parcourir les fichiers à déplacer
    for filename in files_to_move:
        source_file = os.path.join(source_folder, filename)
        destination_file = os.path.join(destination_folder, filename)

        # Vérifier si le fichier existe dans le dossier source
        if os.path.exists(source_file):
            shutil.move(source_file, destination_file)
            print(f"Fichier déplacé : {source_file} -> {destination_file}")
        else:
            print(f"Fichier introuvable : {source_file}")

In [ ]:
move_files("","data/data_api", ["results_1996_to_2008.json", "results_2008_to_2022.json" ] )

Après avoir récupéré nos deux fichiers json il faut les convertir en dataframe puis en csv afin de les exploiter par la suite. Nous avons ainsi codé une fonction permettant d'extraire la date du titre des documents, une autre qui les convertit en dataframe en ne gardant qu'un nombre limité d'informations. 

In [13]:
def extract_date_from_title(title):
    """
    Extrait la date d'un titre. Si plusieurs dates sont présentes, retourne la plus récente.

    :param title: Le titre de l'objet.
    :return: La date extraite ou None si aucune date valide n'est trouvée.
    """
    # Regex pour les formats de date
    date_patterns = [
        r"(\d{1,2})\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+(\d{4})",
        r"(\d{1,2}/\d{1,2}/\d{4})"
    ]

    found_dates = []

    for pattern in date_patterns:
        matches = re.findall(pattern, title, re.IGNORECASE)
        for match in matches:
            if len(match) == 3:  # Format "11 mai 2005"
                day, month, year = match
                month_mapping = {
                    "janvier": 1, "février": 2, "mars": 3, "avril": 4, "mai": 5, "juin": 6,
                    "juillet": 7, "août": 8, "septembre": 9, "octobre": 10, "novembre": 11, "décembre": 12
                }
                month_num = month_mapping[month.lower()]
                found_dates.append(datetime(int(year), month_num, int(day)))
            elif len(match) == 1:  # Format "12/12/2014"
                date_str = match[0]
                found_dates.append(datetime.strptime(date_str, "%d/%m/%Y"))

    if found_dates:
        return max(found_dates).strftime("%Y-%m-%d")  # Retourne la date la plus récente au format AAAA-MM-JJ

    return None

In [14]:
def results_to_dataframe(json_data):
    """
    Convertit les données JSON en DataFrame en extrayant des champs spécifiques.

    :param json_data: Les données JSON à analyser.
    :return: Un DataFrame contenant les données extraites.
    """
    data = []

    # Accéder à la liste des résultats
    results = json_data.get("results", [])

    for result in results:
        # Extraire les informations requises
        titles = result.get("titles")
        title_info = titles[0] if isinstance(titles, list) and titles else {}

        title = title_info.get("title")
        id = title_info.get("id")
        date = extract_date_from_title(title)  # Extraire uniquement à partir du titre
        nature = result.get("nature")
        etat = result.get("etat")
        origin = result.get("origin")
        date_publication = result.get("datePublication")

        # Ajouter les données dans la liste
        data.append({
            "Titre": title,
            "ID": id,
            "Date": date,
            "Nature": nature,
            "Etat": etat,
            "Origine": origin,
            "Date Publication": date_publication
        })

    # Créer et retourner un DataFrame
    return pd.DataFrame(data)


Ensuite, nous appliquons ces fonctions à nos deux fichiers json avant de les concaténer sous un unique et de le transformer en csv 

In [15]:
with open("data/data_api/results_1996_to_2008.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

results_1996_to_2008 = results_to_dataframe(json_data)

with open("data/data_api/results_2008_to_2022.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

results_2008_to_2022 = results_to_dataframe(json_data)

results_1996_to_2022 = pd.concat([results_1996_to_2008, results_2008_to_2022], ignore_index=True)

results_1996_to_2022.to_csv("results_LODA.csv", index=False, encoding="utf-8")

Par souci de clarté on déplace les documents dans un dossier spécifique.

In [ ]:
move_files("","data/data_api", ["results_LODA.csv" ] )

### Nettoyage des données de Légifrance <a class="anchor" id="section2"></a>

In [6]:
# Chemin vers le fichier CSV
chemin_fichier = "data/data_api/results_LODA.csv"
df_loda = pd.read_csv(chemin_fichier)

In [ ]:
print(f"Nombre d'observations: {len(df_loda)}")

In [ ]:
# On met la date sous format date de datetime
df_loda['Date'] = pd.to_datetime(df_loda['Date']) 

# On extrait l'année (les 4 premiers caractères) et le mois (caractères à l'index 5 et 6) de la variable Date Publication
df_loda['Année'] = df_loda['Date Publication'].str[:4].astype(int)
df_loda['Mois'] = df_loda['Date Publication'].str[5:7].astype(int)

df_loda.head()

In [ ]:
# Vérification pertinence des données
print("Valeurs uniques dans la colonne 'Année' :")
print(df_loda['Année'].unique())

print("\nValeurs uniques dans la colonne 'Mois' :")
print(df_loda['Mois'].unique())

print("\nValeurs uniques et leur fréquence dans la colonne 'Nature' :")
df_loda['Nature'].value_counts()


On se rend compte d'une incohérence : certaines données présentent une date de publication en 2999. Pour pallier ce problème, on se permet d'utiliser la variable Date (souvent différentes de quelques jours à peine, donc peu problématique pour notre analyse réalisée au plus à l'échelle mensuelle) afin de compléter les valeurs des variables 'Mois' et 'Année'.

In [ ]:
# On règle l'incohérence pour les années de publication en 2999 (on utilise la ariable Date)
print(f"Nombre d'incohérences avant traitement (doit être positif): {len(df_loda[df_loda['Année'] == 2999])}")

df_loda.loc[df_loda['Année'] == 2999, 'Année'] = df_loda.loc[df_loda['Année'] == 2999, 'Date'].dt.year
df_loda.loc[df_loda['Année'] == 2999, 'Mois'] = df_loda.loc[df_loda['Année'] == 2999, 'Date'].dt.month

print(f"Nombre d'incohérences après traitement (doit être nul): {len(df_loda[df_loda['Année'] == 2999])}")


On se permet également d'exclure, pour le reste de l'étude, les nouvelles normes législatives adoptées de type "décision" puisqu'elles ne sont qu'au nombre de 3 sur la période étudiée (trop faible et donc sûrement pas significatif pour le reste de l'analyse).

In [11]:
df_loda = df_loda[df_loda['Nature'] != 'DECISION']

Pour l'affichage des figures dans le main, nous utilisons souvent la fonction nommée tri_occurrence, définie dans le script Python. Cette fonction prend en entrée un dataframe (ici df_loda) et le transforme en un dataframe qui rend compte des occurrences de publication des textes de loi, en fonction de leur type, mois et année de publication.

### Sauvegarde des tableaux de données finalisées <a class="anchor" id="section3"></a>

In [12]:
# Information requise pour la connexion au Bucket de Anh Linh sur le MinIO Client cloud
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "anhlinh"
fs.ls(MY_BUCKET)

# Export du DataFrame dans S3
FILE_PATH_OUT_S3_LODA = f"{MY_BUCKET}/diffusion/df_loda.csv"
with fs.open(FILE_PATH_OUT_S3_LODA, "w") as file_out_loda:
    df_loda.to_csv(file_out_loda)

In [ ]:
fs.ls(f"{MY_BUCKET}/diffusion")